# NLP 05: The importance of deduplication

Real world free text is messy. Free text is basically giving every data entry contributor permission to give you a unique snowflake amalgamation of words, spelled in whatever way just happens to come out. Once that data is collected, it then has to be turned into something relevant.

A large part of this is ensuring that each row only represents one "thing." When trying to determine how much you have of something, like apples, it doesn't do any good to count the same one over and over again. The same is true of datasets, large and small. If the same item is represented multiple times it will create problems in the analysis and make any insights much less insightful.

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
import re
from time import gmtime, strftime
import sys
import os
import io

import zipfile

In [19]:
with zipfile.ZipFile("../aws/data/full-oldb.20220708.zip", 'r') as zip_ref:
    zip_ref.extractall("data/original")
    
df = pd.read_csv('data/original/nodes-addresses.csv')

C:\Users\jfish\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (2,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


The datasets housed by the [International Consortium of Investigative Journalists](https://www.icij.org/), such as the Panama Papers, include some excellent examples of such real world free text data. In particular, the addresses in the [Offshore Leaks dataset](https://offshoreleaks.icij.org/) are particularly good. The initial dataset has 402,246 rows and 225 country codes. However, there are 365 countries, so there's something fishy in this dataset.

In [20]:
print('Number of rows in the dataset:',df.shape[0])
print(f'Number of country codes in the dataset: {df.country_codes.nunique()}')
print('Number of countries in the dataset: {}'.format(df.countries.nunique()))

Number of rows in the dataset: 402246
Number of country codes in the dataset: 225
Number of countries in the dataset: 365


I don't have to dig much deeper to find more inconsistencies. Using `pandas` built in duplicate finder, I can see not only that there are duplicate addresses, but there are addresses in the `name` column, the use of punctuation is all over the place, and there is no standardization in the format of the address.

The duplicate entries for "1 Cabot Square" are a good example:

- "1 Cabot Square; London E14 4QJ; United Kingdom"
- 1 Cabot Square; London; United Kingdom E14 4QR

In not being from the UK, it's impossible for me to tell if the "E14" portion of the address is a street number, apartment/flat number, or some kind of postal code.

In order to analyze this data and get meaningful results, this data needs to be cleaned and standardized. Most importantly, the same address needs to point to a single observation (i.e.: one row per address).

In [21]:
pd.set_option('display.max_colwidth', 1000)

In [22]:
df[df.address.duplicated(keep=False)].sort_values('address').head(31)

,node_id,address,name,countries,country_codes,sourceID,valid_until,note
209760,81013976,"""Chequers""","""Chequers""; 1741, Tiki Tiki Road, RD 1; Te Kuiti 3981; New Zealand",New Zealand,NZL,Paradise Papers - Appleby,Appleby data is current through 2014,NaN
155649,81021301,"""Chequers""","""Chequers""; Tiki Tiki Road; R.D.1 Te Kuiti; New Zealand",New Zealand,NZL,Paradise Papers - Appleby,Appleby data is current through 2014,NaN
175526,81047974,"""Seascape""","""Seascape"" ; 27 Devon Point Lane; Devonshire FL05; Bermuda",Bermuda,BMU,Paradise Papers - Appleby,Appleby data is current through 2014,NaN
173661,81045646,"""Seascape""","""Seascape""; 20 Grape Bay Drive; Paget PG 06; Bermuda",Bermuda,BMU,Paradise Papers - Appleby,Appleby data is current through 2014,NaN
158234,81026137,"#05-06, Pan Pacific Serviced Suites","#05-06, Pan Pacific Serviced Suites; 96 Somerset Road; Singapore 238163",Singapore,SGP,Paradise Papers - Appleby,Appleby data is current through 2014,NaN
166520,81036861,"#05-06, Pan Pacific Serviced Suites","#05-06, Pan Pacific Serviced Suites; 96 Somerset Road; Singapore 238163",Singapore,SGP,Paradise Papers - Appleby,Appleby data is current through 2014,NaN
202791,81084819,"#19-01 Ardmore II, Ardmore Park","#19-01 Ardmore II, Ardmore Park; 259947; Singapore",Singapore,SGP,Paradise Papers - Appleby,Appleby data is current through 2014,NaN
202790,81084818,"#19-01 Ardmore II, Ardmore Park","#19-01 Ardmore II, Ardmore Park; 259947; Singapore",Singapore,SGP,Paradise Papers - Appleby,Appleby data is current through 2014,NaN
86870,14086865,-,NaN,NaN,NaN,Panama Papers,The Panama Papers data is current through 2015,NaN
207173,81009999,-,-; NONE,NaN,NaN,Paradise Papers - Appleby,Appleby data is current through 2014,NaN


## Standardizations

To be honest, the amount of cleaning the `address` column needs is a little overwhelming. I assumed this had to be a solved problem, what with the international mail system and all. However, I couldn't find a solution that would be able to process the addresses from all the different countries represented in this dataset. There are some solutions for US addresses that I may apply to just that country, but for the most part I'll need to write code for my purposes. The best solution I could find for this problem is to parse the data into more manageble and meaningful chunks.

To get started, I literally went through by hand and did some editing. This was obviously too tedious to keep my attention for long (I got through the country, islands, and cities of the Bahamas). While not sustainable for all 400,000+ rows, it did give me an initial feel for the data.

This manual parsing revealed two large tasks that need to be complete on the full dataset regardless of country:

- Ensure full addresses are in the `address` column
- Some `country_codes` have slipped into the `countries` column

## Move full addresses in the `name` column to the `address` column

A quick sanity check shows that I can just look for rows where the `name` column is longer than the `address` column. It is possible that some company or person has a very long name and short address. This doesn't appear to be the case in this dataset though, thank goodness.

In [23]:
df.loc[df['name'].str.len() > df['address'].str.len(), ['address', 'name']]

,address,name
528,6B Chenyu Court,6B Chenyu Court; 22-24 Kennedy Road; Hong Kong
529,15C Suchun Industrial Square,15C Suchun Industrial Square; Suzhou Industrial Park; 215126 Suzhou; People's Republic of China
151607,"8F., No. 68","8F., No. 68; Minfu 13th St. Taoyuan County; 330; Taiwan"
151608,"No. 66, Ln. 20","No. 66, Ln. 20; Dafu Rd., Shengang Township; 429 Taichung County; Taiwan"
151609,"1F., No. 18, Ln. 274","1F., No. 18, Ln. 274; Chang'an St., Luzhou City; Taipei County 247, Taiwan; People's Republic of China"
...,...,...
210828,"285 Central Park West, #2","285 Central Park West, #2; New York; New York 10024; United States of America"
210829,9481 Gatetrail Drive,9481 Gatetrail Drive; Dallas; Texas 75238; United States of America
210830,2822 Millers Run Lane,2822 Millers Run Lane; Lancaster; PA 17601; United States of America
210831,24 Bullimore Grove,24 Bullimore Grove; Kenilworth CV8 2QF; Warwickshire; United Kingdom


### Swap columns

This is a nifty trick I probably picked up from StackOverflow. The logic behind it is the same as multiple value assignment:

`a, b, c = output_that_gives_multiple_vals(1, 2, 3)`

Or value reassignment:

`a, b = b, a`

In [24]:
df.loc[df['name'].str.len() > df['address'].str.len(), ['address', 'name']
      ] = df.loc[df['name'].str.len() > df['address'].str.len(), ['name', 'address']].values

I can't stop there however. When the `address` column is empty, the above logic won't work. A second column value reassignment will take care of this.

In [25]:
df[df['name'].notna() & df['address'].isna()]

,node_id,address,name,countries,country_codes,sourceID,valid_until,note
210833,88000000,NaN,"WINDWARD SIDE , , SABA",NaN,NaN,Paradise Papers - Aruba corporate registry,Aruba corporate registry data is current through 2016,NaN
210834,88000001,NaN,"PISO 3, PUERTA 4, ANDORRA LA VELLA , PRINCIPAT D ANDORRA",Andorra,AND,Paradise Papers - Aruba corporate registry,Aruba corporate registry data is current through 2016,NaN
210835,88000002,NaN,"BLOWING POINT , CUL DE SAC, ANGUILLA",Anguilla,AIA,Paradise Papers - Aruba corporate registry,Aruba corporate registry data is current through 2016,NaN
210836,88000003,NaN,"CARIBBEAN COMMERCIAL CENTER, POSTBUS 20, ANGUILLA BWI",Anguilla,AIA,Paradise Papers - Aruba corporate registry,Aruba corporate registry data is current through 2016,NaN
210837,88000004,NaN,"HEYWOOD HOUSE SOUTH HILL, ANGUILLA, BRITISH WEST INDIES",Anguilla,AIA,Paradise Papers - Aruba corporate registry,Aruba corporate registry data is current through 2016,NaN
...,...,...,...,...,...,...,...,...
390990,68001690,NaN,"Manase, Samoa",Samoa,WSM,Paradise Papers - Samoa corporate registry,Samoa corporate registry data is current through 2016,NaN
390991,68001691,NaN,"Vailele-uta, Faleata Sisifo, Samoa",Samoa,WSM,Paradise Papers - Samoa corporate registry,Samoa corporate registry data is current through 2016,NaN
390992,68001692,NaN,"Lotopa, Falelatai & Samatau, Samoa",Samoa,WSM,Paradise Papers - Samoa corporate registry,Samoa corporate registry data is current through 2016,NaN
390993,68001693,NaN,"Sogi, Vaisigano No.2, Samoa",Samoa,WSM,Paradise Papers - Samoa corporate registry,Samoa corporate registry data is current through 2016,NaN


In [26]:
df.loc[df['name'].notna() & df['address'].isna(), ['address', 'name']
      ] = df.loc[df['name'].notna() & df['address'].isna(), ['name', 'address']].values

## Country name in `countries` column

Some `country_codes` snuck into some of the values for the countries column. This makes up a relatively small portion of the dataset, but effects quiet a few countries.

In [27]:
print('Number of country_codes in the wrong column:', df.loc[df['countries'].str.len()<=3, 'countries'].count())
print('Number of unique country codes in the countries column:', df.loc[df['countries'].str.len()<=3, 'countries'].nunique(), '\n')
display(df.loc[df['countries'].str.len()<=3, 'countries'].unique())

Number of country_codes in the wrong column: 1868
Number of unique country codes in the countries column: 123 



array(['HKG', 'SYC', 'VGB', 'BLZ', 'USA', 'ARE', 'VCT', 'GBR', 'PAN',
       'MUS', 'MLT', 'AIA', 'SGP', 'BHS', 'CYP', 'KNA', 'LCA', 'LVA',
       'GIB', 'MHL', 'CHE', 'ESP', 'QAT', 'PRT', 'WSM', 'CZE', 'THA',
       'ITA', 'BRA', 'ARG', 'ROU', 'FIN', 'FRA', 'POL', 'VNM', 'DNK',
       'SVK', 'IRL', 'CAN', 'TUR', 'VUT', 'DEU', 'RUS', 'EGY', 'NLD',
       'GRC', 'ZAF', 'DZA', 'CHL', 'ISR', 'AUS', 'MCO', 'AGO', 'SRB',
       'IND', 'AND', 'VEN', 'SVN', 'JEY', 'MEX', 'UZB', 'KAZ', 'NOR',
       'PRY', 'BEL', 'BGR', 'JPN', 'UKR', 'SWE', 'NGA', 'TUN', 'PER',
       'HUN', 'ECU', 'IDN', 'PHL', 'LUX', 'KEN', 'SAU', 'PAK', 'BLR',
       'CIV', 'ALB', 'MDG', 'ZMB', 'NZL', 'URY', 'AUT', 'MAR', 'BIH',
       'HRV', 'COL', 'AZE', 'OMN', 'JOR', 'GHA', 'SMR', 'LAO', 'LBN',
       'MDA', 'BMU', 'EST', 'SOM', 'MNG', 'BGD', 'MWI', 'NIC', 'GEO',
       'CUB', 'UGA', 'CYM', 'MYS', 'HND', 'KOR', 'ZWE', 'NAM', 'TZA',
       'BOL', 'LBY', 'COD', 'PSE', 'BHR', 'LTU'], dtype=object)

My geography isn't good enough to know what the country name is for each of the codes, so I'll need a way to grab those values.

To address this, I'm going to use the `country_codes` and `countries` columns to create a dictionary with the codes as the key and the country name as the value. I can then use this dictionary to map the country names into the `countries` column. I find this technique useful for all kinds of data wrangling problems as I often need to replace some kind of abbreviation for the actual value or vice versa.

In [28]:
codes_to_countries = pd.DataFrame(
    df.loc[
        df['countries'].str.len()>3
    ].groupby('country_codes')['countries'
                              ].value_counts()).drop(['countries'], axis=1
                                                    ).reset_index().set_index('country_codes').to_dict()['countries']
codes_to_countries

{'ABW': 'Aruba',
 'AFG': 'Afghanistan',
 'AGO': 'Angola',
 'AIA': 'Anguilla',
 'ALB': 'Albania',
 'AND': 'Andorra',
 'ANT': 'Netherlands Antilles',
 'ARE': 'United Arab Emirates',
 'ARG': 'Argentina',
 'ARM': 'Armenia',
 'ASM': 'American Samoa',
 'ATG': 'Antigua and Barbuda',
 'AUS': 'Australia',
 'AUT': 'Austria',
 'AUT;LIE': 'Austria;Liechtenstein',
 'AZE': 'Azerbaijan',
 'BEL': 'Belgium',
 'BEN': 'Benin',
 'BES': 'Sint Eustatius and Saba Bonaire',
 'BFA': 'Burkina Faso',
 'BGD': 'Bangladesh',
 'BGR': 'Bulgaria',
 'BHR': 'Bahrain',
 'BHS': 'Bahamas',
 'BIH': 'Bosnia and Herzegovina',
 'BLR': 'Belarus',
 'BLZ': 'Belize',
 'BMU': 'Bermuda',
 'BOL': 'Plurinational State of Bolivia',
 'BRA': 'Brazil',
 'BRB': 'Barbados',
 'BRN': 'Brunei Darussalam',
 'BTN': 'Bhutan',
 'BWA': 'Botswana',
 'CAF': 'Central African Republic',
 'CAN': 'Canada',
 'CHE': 'Switzerland',
 'CHL': 'Chile',
 'CHN': 'China',
 'CIV': "Côte d'Ivoire",
 'CMR': 'Cameroon',
 'COD': 'The Democratic Republic of the Congo',


In [29]:
df.loc[df['countries'].str.len()<=3, 'countries'] = df.loc[df['countries'].str.len()<=3, 'countries'].map(codes_to_countries)

## Break into more managable smaller pieces

Lastly, I decided it would be easier to handle the challenges of each country individually. This will let me to take advantage of any formatting similarities and to become familiar with the way different countries tend to enter address data. This also helps processing time, as I'll be working with smaller datasets and can apply lessons learned between datasets.

There are quite a few countries with low representation, so I grouped those together.

In [30]:
country_cts = pd.DataFrame(df['countries'].value_counts(dropna=False)).reset_index()
country_cts.columns = ['country', 'count']
country_cts

,country,count
0,NaN,125327
1,China,32491
2,Hong Kong,28269
3,United States,24365
4,Barbados,19348
...,...,...
238,New Caledonia,1
239,Syrian Arab Republic,1
240,British Virgin Islands;Brunei Darussalam,1
241,Sao Tome and Principe,1


In [31]:
country_cts[country_cts['count'] < 500]

,country,count
66,Malta,473
67,Sweden,422
68,Costa Rica,417
69,Czech Republic,412
70,Kazakhstan,411
...,...,...
238,New Caledonia,1
239,Syrian Arab Republic,1
240,British Virgin Islands;Brunei Darussalam,1
241,Sao Tome and Principe,1


## Save the data

Since I decided to break the data into smaller batches, I need a new folder to save them in.

In [32]:
!mkdir data/edited

The syntax of the command is incorrect.


In [33]:
df[df['countries'].isin(
    country_cts[country_cts['count'] < 500][
        'country'].to_list())].to_csv('data/edited/addresses_low_representation.csv', index=False)

In [34]:
for country in country_cts[country_cts['count'] >= 500]['country'].to_list():
    df[df['countries'] == country].to_csv('data/edited/addresses_{}.csv'.format(country), index=False)